# Lab 2 - Logistic Regression and Model Evaluation      

The purpose of this lab is to:      
(1) Familiarize you with the implementing logistic regression in Scikit Learn.     
(2) Explore different ways to evaluate binary classification models and practice interpreting those evaluations.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import sklearn as sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection

## Logistic Regression        

In this lab, you will use the Von Mises stress to predic the presence of absence of fractures on the surface of the Greenland Ice Sheet. The code block below loads your lab data set and drops any columns with missing data. You will see the following variables in the data set:     

**label** - 0 if no crevasses were detected at that location, 1 if crevasses were detected at that location         
**stress** - the Von Mises stress in Pascals (Pa) at the given location     

The code block then splits your dataset in the train, validate, and test datasets. We will use 70% of data for training, 10% for validation, and 20% for testing. We need a separate validation dataset because we eventually want to tune the detection threshold for crevasses.      
Note that the `LogisticRegression` object takes as input 2D arrays of shape (X,1) where X is the number of data samples. Therefore, we need to use the `to_numpy().reshape(-1,1)` function to get our input variable from a pandas `DataSeries` into a 2D array (e.g. shape goes from (X,) to (X,1)).

In [14]:
data = pd.read_csv("https://raw.githubusercontent.com/rtculberg/ml_in_eas/main/data/CrevasseData2.csv")
data = data.dropna(axis=0, how='any')

x = data['stress'].to_numpy().reshape(-1,1)
y = data['label']

# split training, validation, and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.125,random_state=10)

Run the code block below to plot histograms of the Von Mises stress in crevasses and uncrevassed areas. With your lab group, discuss the following questions:            

How separable do these two categories appear to be? Is there are clear cutoff in stress between crevassed and uncrevassed regions?   

In [ ]:
fig, ax = plt.subplots(layout='constrained')
ax.hist(x[y == 0], label="uncrevassed", alpha = 0.5)
ax.hist(x[y == 1], label="crevassed", alpha = 0.5)
ax.set_xlabel('Von Mises Stress (Pa)'); plt.ylabel('Count')
ax.set_title('Data Distributions')
plt.legend()
plt.show()

Check for a class imbalance. How does our sample size for crevassed vs. uncrevassed areas compare?       

**Hint:** Take a look at the `value_counts()` function for a pandas `DataFrame`.

In [ ]:
# Enter code to check for class imbalance


Use `StandardScaler` to apply a z-score transform to your input feature (stress/`x`). Remember to train your scaler on only the training data set and then apply those same statistics to the validation and test sets!

In [18]:
# Add your code here to apply the z-score to the stress data


Add code below instantiate a new `LogisticRegression` object called `regress`. Remember to use `class_weight='balanced'` to account for any class imbalances in the dataset. Then fit the logistic regression model on the scaled training dataset. You may find the documentation for `LogisticRegression` and the associated examples helpful: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html.     

In [ ]:
# Add your code here to train the logistic regression model


In the code block below, we would like to plot the model-predicted probability of fracture as a function of the Von Mises stress. To do so, we can generate an array of evenly spaced stress values between and feed it to the `predict_proba()` function to predict the probability of fracture for each stress value.      

At the top of the code block, generate an array of values from -3 to 5 in increments of 0.05 and call the new variables `stress`. Then use the `predict_proba()` attribute of your `LogisticRegression` object to predict the probability of fracture for each values in this new array. Call this new variable `prob`.     

You should see that `prob` is an (X,2) array (where X is the size of the training dataset). This is because `LogisticRegression` provides predictions for both class labels. So `prob[:,0]` is the probability of no fracture for each stress value and `prob[:,1]` is the probability of fracture for each stress value.     

By running the completed code block you should see a plot comparing the probability of fracture to the distributions of crevassed and uncrevassed points in the original data. The red dashed line marks the default threshold for discriminating between crevassed and uncrevassed areas. It assumes that if we predict that a location has a >=50% probability of fracture, it should be marked as crevassed.

In [ ]:
# Add your code here to generate a new array of normalized stress values between
# -3 and 5 and then predict the probability of fracture on this new array


# Calculate the stress threshold of discriminating between fracture and no fracture
# assuming a default cutoff probability of 50%
threshold = np.argmin(np.abs(prob[:,1] - 0.5))
rescaled_stress = ss.inverse_transform(stress)

# Plot the model predictions vs. the distribution of the raw data
fig, ax = plt.subplots(layout='constrained')
im = ax.hist2d(data['stress']/1000, data['label'], (100,50))
ax.plot(rescaled_stress/1000, prob[:,1], color="white", label="Logistic Regression Fit")
ax.vlines(rescaled_stress[threshold]/1000,0,1, color="red", linestyle="dashed")
ax.plot(rescaled_stress/1000, 0.5*np.ones(stress.shape), color="white", linestyle="dotted")
ax.set_xlabel('Von Mises Stress (kPa)'); plt.ylabel('Probability of Fracture')
ax.set_title('Logistic Regression Model')
cbar = fig.colorbar(im[3], ax=ax, label="Data Count")
plt.legend(loc="right")
plt.show()

## Evaluation Metrics   

In this section of the lab, we will explore different ways to evaluate our model for predicting crevassed regions. In the code blocks below, complete the following tasks.
    
(1) Plot the confusion matrix for the test set. Set `normalize='true'` to list the classification rates, rather than the raw counts. See the documentation for `metrics.confusion_matrix` and `metrics.ConfusionMatrixDisplay` for help.      
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html                 
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html

In [ ]:
# Enter your code here to plot the confusion matrix


(2) The code below lists the recall, precision, and f1 score on the test set. See the documentation for `metrics.precision_recall_fscore_support`.
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html      
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html         

In [ ]:
precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test, y_pred, average='binary')
print("Precision:", precision, "Recall:", recall, "F1 Score:", f1)

(3) The code below plots the ROC curve with the change line and list the area under curve (AUC) metric on the test set. See the documentation for `metrics.RocCurveDisplay`.         
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html

In [ ]:
metrics.RocCurveDisplay.from_estimator(regress,x_test_scaled,y_test, plot_chance_level=True)

## Cross Fold Validation         

As we discussed in class, one of the ways that we can avoid overfitting and ensure that our model evaluation metrics are fully representative of the data is to use cross-fold validation. In the code below, you will explore the difference between using standard stratified K-Fold validation and regionally stratified K-fold validation for our crevasse dataset.      

The first code block applies standard K-fold validation. In this version, we randomly split our data into 10 different test and training datesets, retrain the model 10 times, and evaluate each model on a different test set. The use of `model_selection.StratifiedShuffleSplit` makes sure that each test and training dataset has the same proportion of crevassed and uncrevassed data as the total dataset.

In [ ]:
# Define the evaluation metrics that we want to run on each model iteration
scoring = ['precision_weighted', 'recall_weighted', 'f1_weighted', 'matthews_corrcoef', 'roc_auc']
# Define how our 10 folds, where the test set is 20% of the data and remaining 80% is for training
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
# Run the cross validation and get the resulting evaluation metrics
scores = model_selection.cross_validate(regress, ss.transform(x), y.to_numpy(), cv=cv, scoring=scoring)

# Print the mean evaluation metrics, ply or minus 1 standard deviation
print('Precision=%.3f+/-%.3f, Recall=%.3f+/-%.3f, F-Score=%.3f+/-%.3f, MCC=%.3f+/-%.3f, AUC=%.3f+/-%.3f' %
      (np.mean(scores['test_precision_weighted']), np.std(scores['test_precision_weighted']),
       np.mean(scores['test_recall_weighted']), np.std(scores['test_recall_weighted']),
       np.mean(scores['test_f1_weighted']), np.std(scores['test_f1_weighted']),
       np.mean(scores['test_matthews_corrcoef']), np.std(scores['test_matthews_corrcoef']),
       np.mean(scores['test_roc_auc']), np.std(scores['test_roc_auc'])))

# Plot the F1 score, MCC, and AUC metrics for each fold
fig, ax = plt.subplots(layout='constrained')
ax.plot(scores['test_f1_weighted'], '-o',label="F1 Score")
#ax.plot(scores['test_matthews_corrcoef'], '-o',label="MCC")
ax.plot(scores['test_roc_auc'], '-o',label="AUC")
ax.set_xlabel('Fold Number'); plt.ylabel('Score')
ax.set_title('10-Fold Cross Validation with Split Shuffle')
ax.set_ylim((0,1))
plt.legend()
plt.show()

The second code block applies regional cross-validation. In the provided dataset, the data is ordered by spatial distance from the northern most point in the dataset. So if we define our test and training datasets by position in the original dataset, we can break create test folds that from different regions of the ice sheet. The code below divides the original dataset into 10 consecutive, non-overlapping folds (e.g. fold 1 is indices 0-1000, fold 2 is indices 1001-2000, etc). For iteration of model training the corresponding fold serves as the test set, and the remaining data is used for training. By looking at how the model evaluation varies across these regional folds, we can see if our model performs better in some regions of the ice sheet than in others.

In [ ]:
# Define the index range for each fold
folds = 10
test_len = np.rint(x.shape[0]/folds).astype(int)
test_folds = 9*np.zeros(x.shape)
for i in range(0,folds):
  test_folds[i*test_len:(i+1)*test_len] = i

# Create a predefined split object using these manually generate test fold indices
ps = model_selection.PredefinedSplit(test_folds)

# Define the evaluation metrics that we want to run on each model iteration
scoring2 = ['precision_weighted', 'recall_weighted', 'f1_weighted', 'matthews_corrcoef', 'roc_auc']
# Run the cross validation
scores2 = model_selection.cross_validate(regress, ss.transform(x), y.to_numpy(), cv=ps, scoring=scoring2, return_indices=True)

# Print the mean evaluation metrics, ply or minus 1 standard deviation
print('Precision=%.3f+/-%.3f, Recall=%.3f+/-%.3f, F-Score=%.3f+/-%.3f, MCC=%.3f+/-%.3f, AUC=%.3f+/-%.3f' %
      (np.mean(scores2['test_precision_weighted']), np.std(scores2['test_precision_weighted']),
       np.mean(scores2['test_recall_weighted']), np.std(scores2['test_recall_weighted']),
       np.mean(scores2['test_f1_weighted']), np.std(scores2['test_f1_weighted']),
       np.mean(scores2['test_matthews_corrcoef']), np.std(scores2['test_matthews_corrcoef']),
       np.mean(scores2['test_roc_auc']), np.std(scores2['test_roc_auc'])))

# Plot the F1 score, MCC, and AUC metrics for each fold
fig, ax = plt.subplots(layout='constrained')
ax.plot(scores2['test_f1_weighted'], '-o', label="F1 Score")
#ax.plot(scores2['test_matthews_corrcoef'],'-o', label="MCC")
ax.plot(scores2['test_roc_auc'], '-o',label="AUC")
ax.set_xlabel('Fold Number'); plt.ylabel('Score')
ax.set_title('10-Fold Cross Validation with Regional Folds')
ax.set_ylim((0,1))
plt.legend()
plt.show()

With your lab group, discuss the following questions:         

(1) How do the average model evaluation metrics compare between the random stratified k-fold method and the regional k-fold method?    
(2) If you ran only a a random k-fold validation and did not consider the impact of spatial correlation and regional variations in ice sheet properties, would you over or underestimate the true performance of your model?         
(3) Looking at the regional k-fold method figure, which are there any spatial trends in model performance that you see?         

## Hyperparameter Tuning   

At the beginning of this lab, we assumed that the optimal probability cutoff of discrimination between crevassed and uncrevassed areas was a probability of 50%. However, this is a hyperparmeter that we can tune to try and improve our model performance. There are a variety of ways to pick an "optimal" detection thresholds. Two of the most common are using the J statistic with the ROC curve or the F1 score with recall-precision curve.   

The code block below shows you how to use the J statistic to pick the optimal threshold for crevasse detection and plot the ROC curve with is point. Remember from lecture that the J statistic helps us find the point on the ROC curve that is furthest from chance level.


In [ ]:
# Use our trained model to predict the probabilty of fracture on our validation sset
y_score = regress.predict_proba(x_val_scaled)

# Calculate the false positive rate and true positive rate for a range of different
# discrimination thresholds
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_score[:,1])
# Calculate the J statistic and find it's maximum value
J = tpr - fpr
ind = np.argmax(J)

# Plot the ROC curve
metrics.RocCurveDisplay.from_estimator(regress,x_val_scaled,y_val, plot_chance_level=True)
# Add a point showing where the J-statistic is maximized
plt.scatter(fpr[ind], tpr[ind], color="orange", label="Optimal Threshold - J Statistic")
plt.legend()

# Print out the best probability threshold and the associated J statistic
print('Best Threshold=%f, J Statistic=%.3f' % (thresholds[ind], J[ind]))

The next code block demonstrates how we can use this optimized detection threshold to predict crevassed vs. uncrevassed areas on our test set. We can then calculate our new model evaluation metrics on this (hopefully) improved classification.    

Run the code below, then discuss the following questions with your lab group:       
(1) How did the evaluation metrics change once you applied the new detection threshold? (Compare these results to what you found in the original "Evaluation Metrics" section of this lab.)        
(2) Is the updated model better at predicting the presence or absence of crevasses?   




In [ ]:
# Predict the probability of fracture on the text dataset
prob = regress.predict_proba(x_test_scaled)
# Create a new empty matrix to hold crevassed vs. uncrevassed prediction
y_pred = np.empty(prob.shape[0])
# If the probabilty exceeds the newly defined threshold, set the prediction to 1
y_pred[prob[:,1] >= thresholds[ind]] = 1
y_pred[prob[:,1] < thresholds[ind]] = 0

# Create and plot the foncusion matrix
confusion_matrix = metrics.confusion_matrix(y_test, y_pred, normalize='true')

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()

# Calculate and print the model evaluation metrics
precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test, y_pred, average='binary')
print('Precision=%.3f, Recall=%.3f, F-Score=%.3f' % (precision, recall, f1))

Alternatively, we could use the F1 score and precision-recall curve to optimize our discrimination threshold. The code block below picks an optimal threshold using the F1 score, then classifies the test dataset using this new threshold and displays the evaluation metrics. 

Discuss the following questions with your lab group:       
(1) Are the optimal detection threshold or model evaluation results different when using the F1 score vs. the J statistic to select the detection threshold?     
(2) Which method do you think is most appropriate for this particular use case (detecting crevasses)? Why?

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(y_val, y_score[:,1])

fscore = (2 * precision * recall) / (precision + recall)
fscore[np.isnan(fscore)]=0
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

fig, ax = plt.subplots(layout='constrained')
ax.plot(recall, precision)
ax.scatter(recall[ix], precision[ix], color="orange", label="Optimal Threshold - F1 Score")
ax.set_xlabel('Recall'); plt.ylabel('Precision')
ax.set_title('Precision-Recall Curve')
plt.legend()
plt.show()

prob = regress.predict_proba(x_test_scaled)
y_pred = np.empty(prob.shape[0])
y_pred[prob[:,1] >= thresholds[ix]] = 1
y_pred[prob[:,1] < thresholds[ix]] = 0

confusion_matrix = metrics.confusion_matrix(y_test, y_pred, normalize='true')

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()

precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test, y_pred, average='binary')
print('Precision=%.3f, Recall=%.3f, F-Score=%.3f' % (precision, recall, f1))